<font color="#CC3D3D"><p>
# RecSys Competition Starter Notebook #3
### Model Ensemble

**글로벌 세팅**

In [1]:
import pandas as pd
import numpy as np
import itertools

DEFAULT_USER_COL = 'user_id'
DEFAULT_ITEM_COL = 'item_id'
DEFAULT_RATING_COL = 'rating'
TOP_K = 5

**앙상블 설정**

In [2]:
# 먼저, W12-comp_02_model_tuning.ipynb를 통해 개별 모델 submission 화일을 생성해야 함 !!!

model_files = {
    "UserKNN": "ensemble_UserKNN.csv",
    "EASE": "ensemble_EASE.csv",
}
weights = {"UserKNN": 1, "EASE": 5}

**앙상블 (RRF/BORDA/VOTING)**

In [3]:
# --- 앙상블 방법 선택 ---
ENSEMBLE_METHOD = "BORDA"  # "RRF", "BORDA", "VOTING" 중 선택

# --- 방법별 파라미터 설정 ---
if ENSEMBLE_METHOD == "RRF":
    K_CONSTANT = 60
    TRUNCATE_AT = None  # None이면 전체 사용
    
elif ENSEMBLE_METHOD == "BORDA":
    MAX_RANK = 20  # Borda Count에서 고려할 최대 순위
    TRUNCATE_AT = MAX_RANK
    
elif ENSEMBLE_METHOD == "VOTING":
    TRUNCATE_AT = 20  # Weighted Voting에서 고려할 최대 순위
    
print(f"앙상블 방법: {ENSEMBLE_METHOD}")
if TRUNCATE_AT:
    print(f"각 모델에서 상위 {TRUNCATE_AT}개만 사용\n")

# --- 점수 계산 ---
parts = []
for name, path in model_files.items():
    w = weights.get(name, 1.0)
    if w == 0:  # 가중치 0이면 건너뛰기
        continue
    
    df = pd.read_csv(path, usecols=[DEFAULT_USER_COL, DEFAULT_ITEM_COL])
    
    # 순위 계산 (파일이 정렬되어 있다고 가정)
    df["rank"] = df.groupby(DEFAULT_USER_COL, sort=False).cumcount() + 1
    
    # 순위 제한 (선택적)
    if TRUNCATE_AT is not None:
        df = df[df["rank"] <= TRUNCATE_AT]
    
    # 빈 데이터프레임 체크
    if len(df) == 0:
        print(f"  {name}: 가중치={w}, TRUNCATE_AT={TRUNCATE_AT} 적용 후 데이터 없음")
        continue
    
    # 방법별 점수 계산
    if ENSEMBLE_METHOD == "RRF":
        # RRF: w / (k + rank)
        df["score"] = w / (K_CONSTANT + df["rank"])
        
    elif ENSEMBLE_METHOD == "BORDA":
        # Borda Count: w * (MAX_RANK - rank + 1)
        # 1위는 MAX_RANK점, 2위는 MAX_RANK-1점, ...
        df["score"] = w * (MAX_RANK - df["rank"] + 1)
        
    elif ENSEMBLE_METHOD == "VOTING":
        # Weighted Voting: 각 아이템이 추천되면 해당 모델의 가중치를 점수로 부여
        # 순위와 무관하게 top-K 안에 있으면 동일한 가중치
        df["score"] = w
    
    parts.append(df[[DEFAULT_USER_COL, DEFAULT_ITEM_COL, "score"]])
    n_users = df[DEFAULT_USER_COL].nunique()
    avg_items = len(df) / n_users if n_users > 0 else 0
    print(f"  {name}: 가중치={w}, 사용자당 평균 {avg_items:.1f}개 아이템")

# --- 점수 합산 확인 ---
if not parts:
    raise ValueError("앙상블에 사용 가능한 모델 결과가 없습니다. TRUNCATE_AT 값을 확인하세요.")

# --- 점수 합산 ---
fused = (
    pd.concat(parts, ignore_index=True)                                             # 모든 모델 결과 병합
      .groupby([DEFAULT_USER_COL, DEFAULT_ITEM_COL], as_index=False)["score"].sum() # 아이템별 점수 합산
)

앙상블 방법: BORDA
각 모델에서 상위 20개만 사용

  UserKNN: 가중치=1, 사용자당 평균 20.0개 아이템
  EASE: 가중치=5, 사용자당 평균 20.0개 아이템


**사용자별 Top-K 추출**

In [4]:
topk = (
    fused.sort_values([DEFAULT_USER_COL, "score"], ascending=[True, False]) # 유저별로 정렬하고, 각 유저 안에서 높은 점수(내림차순) 순으로 정렬
         .groupby(DEFAULT_USER_COL, as_index=False)                         # 유저 단위로 그룹화
         .head(TOP_K)                                                       # 각 유저별 상위 K개 추천 선택
         .drop(columns="score")                                             # 점수 컬럼 제거
         .reset_index(drop=True)                                            # 인덱스 리셋
)

**Submission 파일 저장**

In [5]:
weight_str = "_".join([str(weights[model]) for model in weights.keys()])
fname = f"submit_{ENSEMBLE_METHOD}_{weight_str}.csv"

# user_id별로 item_id를 공백으로 연결
top_k = topk.groupby(DEFAULT_USER_COL)[DEFAULT_ITEM_COL].apply(lambda x: ' '.join(x)).reset_index()
top_k.columns = [DEFAULT_USER_COL, 'item_ids']

top_k.to_csv(fname, index=False)
print(f"저장 완료: {fname} (Top-{TOP_K} per user)")

저장 완료: submit_BORDA_1_5.csv (Top-5 per user)


<font color="#CC3D3D"><p>
# End